In [ ]:
import pandas as pd
from io import BytesIO
import requests
from zipfile import ZipFile
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, WMTSTileSource
import numpy as np

output_notebook()

url = "https://simplemaps.com/static/data/world-cities/basic/simplemaps_worldcities_basicv1.75.zip"
response = requests.get(url)
zip_file = ZipFile(BytesIO(response.content))

with zip_file.open("worldcities.csv") as file:
    cities_data = pd.read_csv(file, usecols=["city", "lat", "lng", "country"], nrows=500)

def wgs84_to_web_mercator(lat, lon):
    k = 6378137
    x = lon * (k * np.pi / 180.0)
    y = np.log(np.tan((90 + lat) * np.pi / 360.0)) * k
    return x, y

cities_data["x"], cities_data["y"] = wgs84_to_web_mercator(
    cities_data["lat"], cities_data["lng"]
)

source = ColumnDataSource(data={
    "x": cities_data["x"],
    "y": cities_data["y"],
    "city": cities_data["city"],
    "country": cities_data["country"],
})

tile_provider_url = "https://a.tile.openstreetmap.org/{z}/{x}/{y}.png"
tile_provider = WMTSTileSource(url=tile_provider_url)
p = figure(
    title="World Cities",
    x_axis_type="mercator",
    y_axis_type="mercator",
    width=900,
    height=600,
    tools="pan, wheel_zoom, reset, save",
)
p.add_tile(tile_provider)

p.circle(
    x="x",
    y="y",
    size=6,
    fill_color="blue",
    fill_alpha=0.6,
    line_color=None,
    source=source,
)

hover = HoverTool()
hover.tooltips = [
    ("City", "@city"),
    ("Country", "@country"),
]
p.add_tools(hover)


show(p)
